## Smoothing

Smoothing can help to discover trends that otherwise might be hard to see in raw data. 

In [1]:
%useLatestDescriptors
%use dataframe
%use lets-plot

In [2]:
LetsPlot.getInfo()

Lets-Plot Kotlin API v.4.10.0. Frontend: Notebook with dynamically loaded JS. Lets-Plot JS v.4.6.1.

In [3]:
var mpg_df = DataFrame.readCSV("https://raw.githubusercontent.com/JetBrains/lets-plot-kotlin/master/docs/examples/data/mpg.csv")
mpg_df.head()

untitled,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.800000,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.800000,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.000000,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.000000,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.800000,1999,6,auto(l5),f,16,26,p,compact


### The default smoothing method is `'linear model'` (or `'lm'`)

In [4]:
// val dat = (mpg_df.names.map { Pair(it, mpg_df.get(it).values())}).toMap()
val dat = mpg_df.toMap()

In [5]:
val mpg_plot = letsPlot(dat) {x="displ"; y="hwy"} 
mpg_plot + geomPoint() + geomSmooth(seed = 42)

<path d="M24.638126776775607 133.18270272631514 L24.638126776775607 133.18270272631514 L30.87562722659223 135.54688554434483 L37.11312767640885 137.908609317941 L43.350628126225445 140.26768064153822 L49.58812857604207 142.62388842918767 L55.82562902585869 144.97700235432978 L62.06312947567528 147.32677122620947 L68.3006299254919 149.6729213307861 L74.5381303753085 152.01515477674405 L80.77563082512515 154.35314790332126 L87.01313127494174 156.68654982652663 L93.25063172475836 159.01498122407727 L99.48813217457496 161.33803348681477 L105.72563262439158 163.65526839457021 L111.9631330742082 165.96621850557332 L118.20063352402482 168.27038847734357 L124.43813397384142 170.56725755868862 L130.67563442365804 172.8562835002383 L136.91313487347463 175.13690811640404 L143.15063532329128 177.40856468533278 L149.38813577310788 179.6706872864352 L155.62563622292447 181.92272204159357 L161.86313667274112 184.16414004675852 L168.1006371225577 186.39445156575215 L174.33813757237436 188.61322083037743 L180.5756380221909 190.82008058509373 L186.81313847200755 193.01474537289866 L193.05063892182415 195.19702252283844 L199.2881393716408 197.36681989702893 L205.5256398214574 199.52414969045168 L211.76314027127404 201.66912792448318 L218.00064072109063 203.80196968247753 L224.23814117090723 205.92298053495693 L230.47564162072382 208.0325449273667 L236.7131420705404 210.1311125078134 L242.95064252035706 212.21918343715157 L249.18814297017371 214.29729365984568 L255.42564341999037 216.36600095357775 L261.6631438698069 218.42587236102494 L267.9006443196235 220.47747337980783 L274.1381447694401 222.521359078421 L280.37564521925674 224.5580671367289 L286.6131456690734 226.5881126871961 L292.85064611889 228.61198475616928 L299.0881465687065 230.6301440664558 L305.32564701852317 232.64302195392443 L311.5631474683398 234.6510201625868 L317.80064791815647 236.65451130650197 L324.03814836797306 238.65383981658968 L330.2756488177896 240.64932322158023 L336.51314926760625 242.6412536420174 L342.7506497174229 244.629899402857 L348.98815016723955 246.61550669305677 L355.2256506170561 248.59830121951126 L361.4631510668727 250.57848981797662 L367.70065151668933 252.55626199566777 L373.938151966506 254.53179138947925 L380.17565241632263 256.5052371307597 L386.41315286613917 258.47674511270895 L392.65065331595576 260.4464491601514 L398.88815376577236 262.41447210400145 L405.12565421558907 264.3809267644379 L411.36315466540566 266.3459168478719 L417.60065511522214 268.3095377633815 L423.83815556503885 270.2718773645404 L430.07565601485544 272.23301662257984 L436.31315646467215 274.1930302366762 L442.55065691448874 276.151987186895 L448.7881573643052 278.1099512350049 L455.0256578141219 280.0669813780151 L461.2631582639385 282.0231322589207 L467.5006587137551 283.9784545387698 L473.7381591635718 285.93299523380796 L479.9756596133883 287.88679802111284 L486.2131600632049 289.8399035158143 L492.4506605130216 291.79234952269286 L498.6881609628382 293.74417126468194 L504.9256614126549 295.69540159054236 L511.1631618624714 297.64607116375527 L517.400662312288 299.5962086344698 L517.400662312288 325.5 L511.1631618624714 323.10763630916523 L504.9256614126549 320.715804720829 L498.6881609628382 318.3245338851401 L492.4506605130216 315.9338544655799 L486.2131600632049 313.54379931090926 L479.9756596133883 311.1544036440614 L473.7381591635718 308.76570526981715 L467.5006587137551 306.3777448033059 L461.2631582639385 303.9905659216058 L455.0256578141219 301.60421564096214 L448.7881573643052 299.21874462242306 L442.55065691448874 296.8342075089838 L436.31315646467215 294.45066329765336 L430.07565601485544 292.06817575020045 L423.83815556503885 289.6868138466906 L417.60065511522214 287.30665228630016 L411.36315466540566 284.9277720402606 L405.12565421558907 282.5502609621454 L398.88815376577236 280.17421446103253 L392.65065331595576 277.7997362433333 L386.41315286613917 275.4269391292265 L380.17565241632263 273.05594594962656 L373.938151966506 270.6868905293577 L367.70065151668

### `LOESS` model does seem to better fit MPG data than linear model.

In [6]:
mpg_plot + geomPoint() + statSmooth(method="loess", size=1.0, seed = 42)

<path d="M24.638126776775607 113.75560313832113 L24.638126776775607 113.75560313832113 L30.87562722659223 117.87077018876494 L37.11312767640885 121.97290832367884 L43.350628126225445 126.05231754996095 L49.58812857604207 130.10297298310724 L55.82562902585869 134.13040193313066 L62.06312947567528 138.1386078403238 L68.3006299254919 142.12934565189022 L74.5381303753085 146.10600728491062 L80.77563082512515 150.0722130538914 L87.01313127494174 154.02806008300558 L93.25063172475836 157.96628595560657 L99.48813217457496 161.87901153306427 L105.72563262439158 165.78529702874692 L111.9631330742082 169.72685543280727 L118.20063352402482 173.5937155640512 L124.43813397384142 177.21215476037503 L130.67563442365804 180.45966322458577 L136.91313487347463 183.4832355417089 L143.15063532329128 186.4441393540196 L149.38813577310788 189.41663637093436 L155.62563622292447 192.46503536845788 L161.86313667274112 195.5466913303231 L168.1006371225577 198.57340546743112 L174.33813757237436 201.6438326310237 L180.5756380221909 204.8966825803088 L186.81313847200755 208.26365448449843 L193.05063892182415 211.1924539224587 L199.2881393716408 213.9669434831552 L205.5256398214574 217.10547787438975 L211.76314027127404 220.34501949945428 L218.00064072109063 223.49691135891362 L224.23814117090723 226.5626487691256 L230.47564162072382 229.68210472466083 L236.7131420705404 232.7902734977909 L242.95064252035706 235.8290837397745 L249.18814297017371 238.85537402166068 L255.42564341999037 241.72912849029424 L261.6631438698069 244.23773564580017 L267.9006443196235 246.2378449665054 L274.1381447694401 247.8955255536519 L280.37564521925674 249.4633702770469 L286.6131456690734 251.1093275856636 L292.85064611889 252.69797069368252 L299.0881465687065 254.026174043035 L305.32564701852317 254.9876348118916 L311.5631474683398 255.74317599516885 L317.80064791815647 256.3636328242311 L324.03814836797306 256.8251320422117 L330.2756488177896 257.1020074586694 L336.51314926760625 257.16996116622 L342.7506497174229 257.1569335045939 L348.98815016723955 257.47913153199426 L355.2256506170561 258.5670400382452 L361.4631510668727 259.78307948536985 L367.70065151668933 259.374738940348 L373.938151966506 258.612468997825 L380.17565241632263 258.5783427441903 L386.41315286613917 258.6197981409011 L392.65065331595576 257.99872697828414 L398.88815376577236 256.65974039854973 L405.12565421558907 255.13990124773503 L411.36315466540566 253.54070827535156 L417.60065511522214 251.85261448574627 L423.83815556503885 250.08417460405843 L430.07565601485544 248.29557641807898 L436.31315646467215 246.51198929175055 L442.55065691448874 244.73017070755816 L448.7881573643052 242.95489141410485 L455.0256578141219 241.18274424986998 L461.2631582639385 239.4046874831192 L467.5006587137551 237.61158883125205 L473.7381591635718 235.79436535185027 L479.9756596133883 233.94859597348741 L486.2131600632049 232.0774650562526 L492.4506605130216 230.1848430137517 L498.6881609628382 228.27458678185263 L504.9256614126549 226.35054086922094 L511.1631618624714 224.4165383245828 L517.400662312288 222.47640162637668 L517.400662312288 273.77658577175 L511.1631618624714 274.76554751897976 L504.9256614126549 275.75328614240726 L498.6881609628382 276.736266843101 L492.4506605130216 277.71096539241705 L486.2131600632049 278.6738690216506 L479.9756596133883 279.6214773892995 L473.7381591635718 280.5503036315995 L467.5006587137551 281.4575761726294 L461.2631582639385 282.3481605697399 L455.0256578141219 283.23161861891845 L448.7881573643052 284.1175986966479 L442.55065691448874 285.0156989542404 L436.31315646467215 285.9299265990445 L430.07565601485544 286.8561580678787 L423.83815556503885 287.79833392044253 L417.60065511522214 288.73203759503485 L411.36315466540566 289.5978929265118 L405.12565421558907 290.38821980654575 L398.88815376577236 291.1135070669497 L392.65065331595576 291.6732692056003 L386.41315286613917 291.5315320812399 L380.17565241632263 290.744852468772 L373.938151966506 290.05258898686395 L367.70065151668

### Applying smoothing to groups

Let's map the vehicle `drivetrain type` (variable 'drv') to the color of points.

This makes it easy to see that points with the same type of the drivetrain are forming some kind of groups or clusters. 

In [7]:
mpg_plot + geomPoint {color="drv"} +
           statSmooth(method="loess", size=1.0, seed = 42) {color="drv"}

<path d="M22.003967065926958 100.66100135901053 L22.003967065926958 100.66100135901053 L25.820876496022123 104.08229031981978 L29.63778592611729 107.47239167428711 L33.454695356212454 110.80875865447558 L37.27160478630762 114.06878289678042 L41.08851421640276 117.23106067344088 L44.90542364649792 120.28596285593636 L48.72233307659309 123.23314588815606 L52.539242506688254 126.11681712383296 L56.35615193678342 129.01431240374825 L60.17306136687856 131.95935523550224 L63.98997079697372 134.93616872397587 L67.80688022706889 137.92629791822753 L71.62378965716405 140.91119915155207 L75.44069908725919 143.87470149633054 L79.25760851735436 146.8062221110801 L83.07451794744952 149.69588887911829 L86.89142737754472 152.53375695680762 L90.70833680763985 155.28804338800887 L94.52524623773499 157.81570542892877 L98.34215566783018 159.969076309184 L102.15906509792532 161.84029091951027 L105.97597452802046 163.63400840421235 L109.79288395811565 165.55564944313437 L113.60979338821082 167.8005146966151 L117.42670281830596 170.34198246077307 L121.24361224840112 172.93959154929377 L125.06052167849629 175.3596662179159 L128.87743110859142 177.38980596915877 L132.6943405386866 178.81947643640734 L136.51124996878175 179.4386149146988 L140.32815939887695 179.16865982044396 L144.1450688289721 178.3119680129409 L147.96197825906725 177.2069254001434 L151.77888768916242 176.04941701680315 L155.59579711925758 174.99683740444414 L159.41270654935275 174.2067432275752 L163.22961597944786 173.82301439803143 L167.04652540954308 173.84855016412382 L170.8634348396382 174.20367279650105 L174.68034426973335 174.80765676212764 L178.49725369982852 175.57935321991687 L182.31416312992368 176.4249059516872 L186.13107256001885 177.23602809624361 L189.94798199011402 177.96170215275708 L193.76489142020918 178.6502589492933 L197.5818008503043 179.35965577624904 L201.3987102803995 180.1477506560197 L205.21561971049462 181.0477390202895 L209.0325291405898 182.00210891122947 L212.84943857068495 182.93236838943739 L216.66634800078012 183.7599379839657 L220.48325743087528 184.41239819303422 L224.3001668609704 184.87037766003215 L228.11707629106567 185.1412833903201 L231.93398572116078 185.23266293021635 L235.7508951512559 185.1520021510994 L239.5678045813511 184.90673011693406 L243.38471401144622 184.50422368663948 L247.2016234415414 183.95181182801707 L251.0185328716366 183.2567796335656 L254.83544230173172 182.42637203837347 L258.6523517318269 181.46779724721358 L262.4692611619221 180.38822988265082 L266.28617059201713 179.19481386895296 L270.1030800221123 177.89466506830354 L273.9199894522076 176.4948736865955 L277.7368988823026 175.00250646619708 L281.5538083123979 173.42460868273054 L285.37071774249296 171.76820596224127 L289.1876271725881 170.0403059342721 L293.0045366026834 168.2478997353852 L296.82144603277845 166.3979633766429 L300.6383554628736 164.4974589875211 L304.4552648929688 162.55333594770894 L308.27217432306395 160.57253191726858 L312.0890837531591 158.5619737747017 L315.9059931832543 156.5285784715989 L319.72290261334945 154.4792538117452 L323.5398120434446 152.42089916180737 L323.5398120434446 250.67303996538297 L319.72290261334945 251.0573779478007 L315.9059931832543 251.43718907039536 L312.0890837531591 251.80581584992692 L308.27217432306395 252.15660924027307 L304.4552648929688 252.4829293431175 L300.6383554628736 252.7781461870548 L296.82144603277845 253.03564058223995 L293.0045366026834 253.24880505845516 L289.1876271725881 253.41104489526916 L285.37071774249296 253.51577925383728 L281.5538083123979 253.55644242081456 L277.7368988823026 253.52648517583685 L273.9199894522076 253.41937629504227 L270.1030800221123 253.22860420414617 L266.28617059201713 252.9476787956097 L262.4692611619221 252.5701334254188 L258.6523517318269 252.08952710584973 L254.83544230173172 251.4994469112633 L251.0185328716366 250.79351061431743 L247.2016234415414 249.9653695698778 L243.38471401144622 249.0087118631257 L239.5678045813511 247.91726573665304 L235.7508951512559 246.68

### Apply linear model with 2nd degree polynomial.

As `LOESS` prediction looks a bit weird let's try 2nd degree polinomial regression.

In [8]:
mpg_plot + geomPoint {color="drv"} +
           statSmooth(method="lm", deg=2, size=1.0, seed = 42) {color="drv"}

<path d="M22.003967065926958 75.39204722829021 L22.003967065926958 75.39204722829021 L25.820876496022123 79.93979014312316 L29.63778592611729 84.40916204162852 L33.454695356212454 88.7994664646726 L37.27160478630762 93.10994623336347 L41.08851421640276 97.33977843343763 L44.90542364649792 101.48806924284906 L48.72233307659309 105.55384869760047 L52.539242506688254 109.5360655296638 L56.35615193678342 113.43358225895184 L60.17306136687856 117.2451707795276 L63.98997079697372 120.9695087487446 L67.80688022706889 124.60517716581785 L71.62378965716405 128.1506596107028 L75.44069908725919 131.60434369983068 L79.25760851735436 134.96452539358557 L83.07451794744952 138.22941684861007 L86.89142737754472 141.3971585287918 L90.70833680763985 144.4658362506512 L94.52524623773499 147.4335037178979 L98.34215566783018 150.29821087331487 L102.15906509792532 153.0580380477578 L105.97597452802046 155.71113541412404 L109.79288395811565 158.25576667933126 L113.60979338821082 160.69035531929217 L117.42670281830596 163.01353106049288 L121.24361224840112 165.22417384020656 L125.06052167849629 167.32145224460544 L128.87743110859142 169.3048535193784 L132.6943405386866 171.17420271255696 L136.51124996878175 172.92966931750993 L140.32815939887695 174.57176083635528 L144.1450688289721 176.1013038256292 L147.96197825906725 177.51941404108288 L151.77888768916242 178.8274581115645 L155.59579711925758 180.027009642908 L159.41270654935275 181.11980275379955 L163.22961597944786 182.10768581744122 L167.04652540954308 182.99257771420406 L170.8634348396382 183.77642830030805 L174.68034426973335 184.46118416922678 L178.49725369982852 185.04876020639028 L182.31416312992368 185.5410169639519 L186.13107256001885 185.93974353202282 L189.94798199011402 186.24664535432382 L193.76489142020918 186.4633363137475 L197.5818008503043 186.59133437406098 L201.3987102803995 186.6320600839873 L205.21561971049462 186.5868373076599 L209.0325291405898 186.45689562356745 L212.84943857068495 186.24337391974416 L216.66634800078012 185.94732479740466 L220.48325743087528 185.56971947316237 L224.3001668609704 185.11145293863382 L228.11707629106567 184.5733491946229 L231.93398572116078 183.9561664253489 L235.7508951512559 183.26060201712772 L239.5678045813511 182.48729735668076 L243.38471401144622 181.63684236805597 L247.2016234415414 180.7097797652101 L251.0185328716366 179.70660901071784 L254.83544230173172 178.62778998079438 L258.6523517318269 177.47374634365588 L262.4692611619221 176.24486866285991 L266.28617059201713 174.94151724020662 L270.1030800221123 173.5640247144632 L273.9199894522076 172.1126984329468 L277.7368988823026 170.58782261310841 L281.5538083123979 168.9896603109172 L285.37071774249296 167.31845521218816 L289.1876271725881 165.5744332621499 L293.0045366026834 163.75780414758555 L296.82144603277845 161.8687626448679 L300.6383554628736 159.90748984618352 L304.4552648929688 157.87415427523837 L308.27217432306395 155.76891290276814 L312.0890837531591 153.5919120712652 L315.9059931832543 151.3432883374747 L319.72290261334945 149.02316924042066 L323.5398120434446 146.63167400198734 L323.5398120434446 243.4866387029586 L319.72290261334945 244.22792231327438 L315.9059931832543 244.9022688765545 L312.0890837531591 245.50979761468346 L308.27217432306395 246.05063606668526 L304.4552648929688 246.52492078930513 L300.6383554628736 246.93279812503525 L296.82144603277845 247.27442504461146 L293.0045366026834 247.5499700717395 L289.1876271725881 247.75961429860624 L285.37071774249296 247.90355250158436 L281.5538083123979 247.981994367457 L277.7368988823026 247.99516584145263 L273.9199894522076 247.94331060938646 L270.1030800221123 247.82669172722737 L266.28617059201713 247.6455934124266 L262.4692611619221 247.40032301230121 L258.6523517318269 247.09121316561854 L254.83544230173172 246.71862417417842 L251.0185328716366 246.28294660153867 L247.2016234415414 245.78460411591539 L243.38471401144622 245.2240565935238 L239.5678045813511 244.60180349693854 L235.7508951512559 243.91838754011

## Using `asDiscrete()` function with numeric data series

In the previous examples we were using a discrete (or categorical) variable `'drv'` to split the data into a groups.

Now let's try to use a numeric variable `'cyl'` for the same purpose.

In [9]:
mpg_plot + geomPoint {color="cyl"} +
           geomSmooth(method="lm", deg=2, size=1.0, seed = 42) {color="cyl"}

<path d="M22.36672185935734 79.75665909969155 L22.36672185935734 79.75665909969155 L28.02918308957439 86.52642009673997 L33.691644319791436 93.1731522380291 L39.354105550008484 99.69663390226606 L45.01656678022553 106.09662630385947 L50.67902801044258 112.37287206121192 L56.34148924065963 118.5250936605413 L62.003950470876646 124.55299181459725 L67.6664117010937 130.45624371821106 L73.32887293131077 136.23450120610102 L78.99133416152782 141.88738882294228 L84.65379539174486 147.41450182166795 L90.31625662196188 152.81540411355172 L95.97871785217893 158.08962620314156 L101.64117908239601 163.23666315285374 L107.30364031261306 168.25597263626042 L112.9661015428301 173.14697315601 L118.62856277304712 177.9090425219473 L124.29102400326417 182.54151670717363 L129.95348523348125 187.04368922396068 L135.6159464636983 191.4148111865801 L141.27840769391534 195.6540922525279 L146.94086892413236 199.76070265479385 L152.6033301543494 203.73377655227435 L158.2657913845665 207.57241692869647 L163.9282526147835 211.27570225719782 L169.59071384500055 214.84269511222377 L175.2531750752176 218.27245284713337 L180.91563630543465 221.56404036073832 L186.57809753565175 224.71654484763056 L192.24055876586874 227.72909226878335 L197.9030199960858 230.60086509966732 L203.56548122630284 233.33112072884435 L209.22794245651988 235.91920971269616 L214.89040368673693 238.36459296760194 L220.55286491695398 240.66685692533787 L226.21532614717103 242.82572571110362 L231.87778737738813 244.84106953576168 L237.54024860760512 246.7129087189599 L243.2027098378221 248.44141305610313 L248.86517106803922 250.02689657465103 L254.52763229825626 251.46980805194673 L260.19009352847337 252.77071794683764 L265.85255475869036 253.93030259885376 L271.51501598890735 254.94932665417784 L277.17747721912446 255.8286246857386 L282.8399384493415 256.5690828983114 L288.5023996795586 257.171621670935 L294.1648609097756 257.6371795146137 L299.8273221399926 257.96669883841787 L305.4897833702097 258.16111374220674 L311.15224460042674 258.2213399032943 L316.81470583064385 258.14826650487 L322.47716706086084 257.9427500676994 L328.1396282910778 257.60560999135026 L333.80208952129493 257.13762558227705 L339.464550751512 256.53953433789303 L345.1270119817291 255.8120312625275 L350.7894732119461 254.95576900783487 L356.45193444216306 253.97135865256325 L362.11439567238017 252.85937096142368 L367.7768569025972 251.62033798781394 L373.4393181328143 250.25475490887086 L379.10177936303126 248.7630820028432 L384.7642405932483 247.14574669767555 L390.4267018234654 245.40314563586944 L396.08916305368246 243.5356467142119 L401.75162428389956 241.54359106808076 L407.41408551411644 239.42729497899583 L413.07654674433354 237.1870516912119 L418.73900797455065 234.8231331286999 L424.40146920476775 232.33579150813887 L430.06393043498474 229.725260846751 L435.72639166520173 226.99175836618434 L441.38885289541884 224.13548579534017 L447.0513141256358 221.15663057621413 L452.71377535585293 218.0553669775772 L458.37623658607004 214.83185712177573 L464.0386978162869 211.48625193014044 L469.701159046504 208.01869199253724 L469.701159046504 258.1598097611402 L464.0386978162869 260.697685409496 L458.37623658607004 263.1184063827989 L452.71377535585293 265.42211327091513 L447.0513141256358 267.6089569951753 L441.38885289541884 269.6790996779257 L435.72639166520173 271.6327155879376 L430.06393043498474 273.4699921672063 L424.40146920476775 275.19113114463323 L418.73900797455065 276.7963497418664 L413.07654674433354 278.2858819761281 L407.41408551411644 279.65998006409745 L401.75162428389956 280.91891592974514 L396.08916305368246 282.06298281732586 L390.4267018234654 283.09249700835994 L384.7642405932483 284.00779963822464 L379.10177936303126 284.80925860370735 L373.4393181328143 285.4972705473093 L367.7768569025972 286.0722628969756 L362.11439567238017 286.5346959309544 L356.45193444216306 286.8850648263828 L350.7894732119461 287.12390163665873 L345.1270119817291 287.251777126493 L339.464550751512 287.2693023746

Easy to see that the data wasn't split into groups. 
`Lets-Plot` offers two solutions in this situation:

 * Use the `group` aesthetic
 * Use the `asDiscrete()` function

The **group** aesthetic helps to create a groups.

In [10]:
mpg_plot + geomPoint {color="cyl"} +
           geomSmooth(method="lm", deg=2, size=1.0, seed = 42) {color="cyl"; group="cyl"}

<path d="M22.36672185935734 39.472030186520556 L22.36672185935734 39.472030186520556 L23.52018618403119 40.14080124140338 L24.67365050870501 40.85675680591356 L25.827114833378857 41.61920229335328 L26.980579158052734 42.42739652123322 L28.134043482726554 43.2805485070549 L29.287507807400402 44.177814104990375 L30.44097213207425 45.118292494888465 L31.59443645674807 46.101022541369844 L32.74790078142192 47.12497904864381 L33.90136510609577 48.189068946308396 L35.054829430769615 49.29212745308183 L36.208293755443464 50.432914279368845 L37.36175808011731 51.61010994606289 L38.51522240479113 52.822312316180046 L39.66868672946498 54.06803345790428 L40.82215105413883 55.34569698233281 L41.97561537881268 56.653636026358186 L43.129079703486525 57.99009208011944 L44.28254402816037 59.35321488830209 L45.43600835283422 60.74106368375425 L46.58947267750804 62.15161003828496 L47.74293700218189 63.58274263627095 L48.89640132685571 65.03227428823442 L50.04986565152956 66.49795149953951 L51.203329976203406 67.97746688891851 L52.35679430087728 69.46847470754648 L53.5102586255511 70.96860963698731 L54.66372295022495 72.4755089397105 L55.81718727489877 73.98683789715585 L56.97065159957262 75.50031829870227 L58.12411592424647 77.0137595456352 L59.277580248920316 78.52509171748153 L60.431044573594164 80.0323997292171 L61.584508898267984 81.53395750676628 L62.73797322294183 83.02826094788992 L63.89143754761568 84.51405833936218 L65.04490187228953 85.99037688960209 L66.19836619696338 87.45654412219864 L67.3518305216372 88.9122030634673 L68.50529484631105 90.35732043763 L69.6587591709849 91.79218743545397 L70.81222349565874 93.21741301528903 L71.96568782033259 94.6339100924194 L73.11915214500644 96.04287533604912 L74.27261646968029 97.44576359045197 L75.42608079435414 98.84425814460275 L76.57954511902796 100.24023818173362 L77.7330094437018 101.63574474774731 L78.88647376837565 103.03294649702337 L80.0399380930495 104.43410632507812 L81.19340241772335 105.84154980287644 L82.34686674239717 107.25763611060836 L83.50033106707102 108.68473195016963 L84.65379539174486 110.12518871175854 L85.80725971641871 111.58132299214338 L86.96072404109256 113.05540041649266 L88.11418836576641 114.5496226041214 L89.26765269044026 116.06611703979388 L90.42111701511408 117.60692956278018 L91.57458133978793 119.17401916103995 L92.72804566446177 120.76925475252386 L93.88150998913562 122.39441364472589 L95.03497431380947 124.05118138281193 L96.18843863848326 125.74115272214385 L97.34190296315711 127.46583348984115 L98.49536728783096 129.22664312986484 L99.64883161250481 131.02491775538698 L100.80229593717866 132.8619135597816 L101.9557602618525 134.73881046282543 L103.10922458652635 136.65671589123838 L104.26268891120017 138.61666861246442 L105.41615323587402 140.61964255762564 L106.56961756054787 142.6665505840404 L107.72308188522172 144.75824813983127 L108.87654620989554 146.89553680316862 L110.03001053456939 149.07916767691438 L111.18347485924323 151.30984462604857 L112.33693918391708 153.58822735056452 L113.49040350859093 155.9149342906668 L113.49040350859093 325.5 L112.33693918391708 320.17385442488046 L111.18347485924323 314.9182811006024 L110.03001053456939 309.73389846737086 L108.87654620989554 304.6213662251781 L107.72308188522172 299.58138823900543 L106.56961756054787 294.6147156117136 L105.41615323587402 289.7221499214737 L104.26268891120017 284.90454661640786 L103.10922458652635 280.1628185538351 L101.9557602618525 275.4979396648765 L100.80229593717866 270.91094871697703 L99.64883161250481 266.40295313685584 L98.49536728783096 261.9751328442899 L97.34190296315711 257.6287440326537 L96.18843863848326 253.36512281511855 L95.03497431380947 249.18568863564636 L93.88150998913562 245.09194732135555 L92.72804566446177 241.08549362760894 L91.57458133978793 237.16801309957174 L90.42111701511408 233.34128304473842 L89.26765269044026 229.60717238105946 L88.11418836576641 225.96764009649414 L86.96072404109256 222.42473203031327 L85.80725971641871 218.98057566728036 L84.6537

The **asDiscrete('cyl')** function will "annotate" the `'cyl'` variable as `discrete`.

This leads to creation of the groups and to assigning of a `discrete` color scale instead of a `continuous`.

In [11]:
mpg_plot + geomPoint {color="cyl"} +
           geomSmooth(method="lm", deg=2, size=1.0, seed = 42) {color=asDiscrete("cyl")}

<path d="M22.003967065926958 39.472030186520556 L22.003967065926958 39.472030186520556 L23.138723923522832 40.14080124140338 L24.27348078111868 40.85675680591356 L25.408237638714553 41.61920229335328 L26.5429944963104 42.42739652123322 L27.677751353906245 43.2805485070549 L28.81250821150212 44.177814104990375 L29.947265069097966 45.118292494888465 L31.082021926693812 46.101022541369844 L32.21677878428969 47.12497904864381 L33.35153564188556 48.189068946308396 L34.48629249948141 49.29212745308183 L35.621049357077254 50.432914279368845 L36.75580621467313 51.61010994606289 L37.890563072268975 52.822312316180046 L39.02531992986485 54.06803345790428 L40.160076787460696 55.34569698233281 L41.29483364505654 56.653636026358186 L42.42959050265242 57.99009208011944 L43.56434736024829 59.35321488830209 L44.69910421784414 60.74106368375425 L45.833861075439984 62.15161003828496 L46.96861793303586 63.58274263627095 L48.103374790631705 65.03227428823442 L49.23813164822758 66.49795149953951 L50.372888505823425 67.97746688891851 L51.5076453634193 69.46847470754648 L52.642402221015146 70.96860963698731 L53.77715907861099 72.4755089397105 L54.91191593620684 73.98683789715585 L56.04667279380271 75.50031829870227 L57.18142965139859 77.0137595456352 L58.316186508994434 78.52509171748153 L59.45094336659031 80.0323997292171 L60.58570022418613 81.53395750676628 L61.720457081782 83.02826094788992 L62.855213939377876 84.51405833936218 L63.98997079697372 85.99037688960209 L65.1247276545696 87.45654412219864 L66.25948451216541 88.9122030634673 L67.39424136976129 90.35732043763 L68.52899822735716 91.79218743545397 L69.66375508495301 93.21741301528903 L70.79851194254888 94.6339100924194 L71.93326880014476 96.04287533604912 L73.0680256577406 97.44576359045197 L74.20278251533648 98.84425814460275 L75.3375393729323 100.24023818173362 L76.47229623052817 101.63574474774731 L77.60705308812402 103.03294649702337 L78.7418099457199 104.43410632507812 L79.87656680331577 105.84154980287644 L81.01132366091159 107.25763611060836 L82.14608051850746 108.68473195016963 L83.2808373761033 110.12518871175854 L84.41559423369918 111.58132299214338 L85.55035109129506 113.05540041649266 L86.6851079488909 114.5496226041214 L87.81986480648678 116.06611703979388 L88.9546216640826 117.60692956278018 L90.08937852167847 119.17401916103995 L91.22413537927434 120.76925475252386 L92.35889223687019 122.39441364472589 L93.49364909446606 124.05118138281193 L94.62840595206188 125.74115272214385 L95.76316280965776 127.46583348984115 L96.8979196672536 129.22664312986484 L98.03267652484948 131.02491775538698 L99.16743338244535 132.8619135597816 L100.3021902400412 134.73881046282543 L101.43694709763707 136.65671589123838 L102.57170395523289 138.61666861246442 L103.70646081282877 140.61964255762564 L104.84121767042464 142.6665505840404 L105.97597452802049 144.75824813983127 L107.11073138561633 146.89553680316862 L108.24548824321218 149.07916767691438 L109.38024510080805 151.30984462604857 L110.51500195840393 153.58822735056452 L111.64975881599977 155.9149342906668 L111.64975881599977 325.5 L110.51500195840393 320.17385442488046 L109.38024510080805 314.9182811006024 L108.24548824321218 309.73389846737086 L107.11073138561633 304.6213662251781 L105.97597452802049 299.58138823900543 L104.84121767042464 294.6147156117136 L103.70646081282877 289.7221499214737 L102.57170395523289 284.90454661640786 L101.43694709763707 280.1628185538351 L100.3021902400412 275.4979396648765 L99.16743338244535 270.91094871697703 L98.03267652484948 266.40295313685584 L96.8979196672536 261.9751328442899 L95.76316280965776 257.6287440326537 L94.62840595206188 253.36512281511855 L93.49364909446606 249.18568863564636 L92.35889223687019 245.09194732135555 L91.22413537927434 241.08549362760894 L90.08937852167847 237.16801309957174 L88.9546216640826 233.34128304473842 L87.81986480648678 229.60717238105946 L86.6851079488909 225.96764009649414 L85.55035109129506 222.42473203031327 L84.41559423369918 218.98057566728036 L83.28083737610

## Effect of `span` parameter on the "wiggliness" the LOESS smoother.

The span is the fraction of points used to fit each local regression.
Small numbers make a wigglier curve, larger numbers make a smoother curve.

In [12]:
import kotlin.math.PI
import kotlin.random.Random

In [13]:
val n = 150
val x_range = generateSequence( -2 * PI ) { it + 4 * PI / n }.takeWhile { it <= 2 * PI }
val y_range = x_range.map{ sin( it ) + Random.nextDouble(-0.5, 0.5) }
val df = mapOf(
    "x" to x_range,
    "y" to y_range
)

In [14]:
val p = ggplot(df) {x="x"; y="y"} + geomPoint(shape=21, fill="yellow", color="#8c564b")
val p1 = p + geomSmooth(method="loess", size=1.5, color="#d62728", seed = 42) + ggtitle("default (span = 0.5)")
val p2 = p + geomSmooth(method="loess", span=.2, size=1.5, color="#9467bd", seed = 42) + ggtitle("span = 0.2")
val p3 = p + geomSmooth(method="loess", span=.7, size=1.5, color="#1f77b4", seed = 42) + ggtitle("span = 0.7")
val p4 = p + geomSmooth(method="loess", span=1, size=1.5, color="#2ca02c", seed = 42) + ggtitle("span = 1")

gggrid(listOf(p1, p2, p3, p4), ncol = 2) + ggsize(800, 600)

<path d="M15.160854049502888 34.39506538523426 L15.160854049502888 34.39506538523426 L18.99904494811122 37.152253810993614 L22.83723584671955 40.00192207429278 L26.675426745327883 42.94415930438241 L30.513617643936186 45.97584726557323 L34.35180854254452 49.090969256297825 L38.18999944115285 52.280892218814685 L42.02819033976118 55.535612838618775 L45.86638123836951 58.84403014913422 L49.70457213697783 62.19515095377299 L53.54276303558616 65.57894845766813 L57.380953934194494 68.98584820402037 L61.21914483280281 72.4071164531255 L65.05733573141114 75.83544596550429 L68.89552663001948 79.26609089160232 L72.73371752862779 82.69878905873139 L76.57190842723611 86.1403639555125 L80.41009932584444 89.61489849489524 L84.24829022445277 93.18991825214033 L88.0864811230611 97.03020290693237 L91.92467202166942 101.33673285624953 L95.76286292027775 105.64869185348866 L99.60105381888609 109.79356731696087 L103.4392447174944 113.6967215719919 L107.27743561610274 117.28532350957835 L111.11562651471105 120.48162909592928 L114.95381741331937 123.21390204186366 L118.7920083119277 125.41783703829955 L122.63019921053603 127.04392365541145 L126.46839010914437 128.0454078501702 L130.3065810077527 128.38285478935967 L134.14477190636103 128.0332307045461 L137.98296280496933 126.99019497274676 L141.82115370357766 125.26525386565304 L145.659344602186 122.8862392073336 L149.49753550079433 119.90405213081713 L153.33572639940266 116.40605515473631 L157.173917298011 112.50187956853168 L161.0121081966193 108.30586865304913 L164.85029909522763 103.93239089167811 L168.68848999383596 99.50521740314377 L172.5266808924443 95.14022475451532 L176.36487179105262 90.9658579435759 L180.20306268966095 87.11026800284857 L184.04125358826929 83.67283405331418 L187.8794444868776 80.72155948748926 L191.71763538548592 78.30318698586919 L195.55582628409425 76.44601829466355 L199.39401718270256 75.17075058405545 L203.2322080813109 74.49482310341637 L207.07039897991922 74.43646012630083 L210.90858987852755 75.01364867477074 L214.74678077713588 76.23108922196872 L218.5849716757442 78.07398502131798 L222.42316257435252 80.51070609312168 L226.26135347296085 83.48817269946574 L230.09954437156918 86.9272692247772 L233.9377352701775 90.73369560263095 L237.77592616878582 94.80381928909898 L241.61411706739415 99.03987652889931 L245.45230796600248 103.16785819494456 L249.2904988646108 106.76821004670231 L253.12868976321914 110.1269153358616 L256.9668806618275 113.41684448088922 L260.8050715604358 116.69722855424055 L264.64326245904414 119.9840238448108 L268.48145335765247 123.2789672956093 L272.3196442562608 126.57823364635905 L276.1578351548691 129.87444036923353 L279.9960260534774 133.15785954400405 L283.83421695208574 136.41732355010123 L287.67240785069407 139.64096311912147 L291.5105987493024 142.81735582582706 L295.34878964791073 145.93610647967762 L299.18698054651907 148.9876415197745 L303.0251714451274 151.96348798588562 L306.86336234373573 154.8572039132668 L310.70155324234406 157.6646824124653 L314.5397441409524 160.38470648345788 L318.3779350395607 163.01813847603634 L318.3779350395607 184.17914452866592 L314.5397441409524 181.14656966683725 L310.70155324234406 178.03011403835006 L306.86336234373573 174.82907674855932 L303.0251714451274 171.54484801441492 L299.18698054651907 168.18172064230498 L295.34878964791073 164.74633622134522 L291.5105987493024 161.2473821221718 L287.67240785069407 157.6946622528415 L283.83421695208574 154.09881931021982 L279.9960260534774 150.47154167683135 L276.1578351548691 146.82498438573518 L272.3196442562608 143.1706220425912 L268.48145335765247 139.51851223088264 L264.64326245904414 135.87639130517957 L260.8050715604358 132.2484640055897 L256.9668806618275 128.63339998496085 L253.12868976321914 125.01567806252795 L249.2904988646108 121.33653206506565 L245.45230796600248 117.4235873628186 L241.61411706739415 112.99138822341122 L237.77592616878582 108.46004862935665 L233.9377352701775 104.1041697080099 L230.09954437156918 100.02213892969554 L226.26